In [ ]:
import heapq
import pickle

from pyrosm import get_data, OSM
from haversine import haversine

Here we import the necessary libraries. Heapq is for the a* search algorithm, pickle allows us to serialize the graph data, pyrosm parses open street map data, and haversine calculates the distance between two coordinates.

In [ ]:
def a_star_search(seattle_graph, start, end):
    """
    parameters
    seattle_graph: graph data for Seattle
    start: starting vertex in the graph
    end: ending/target vertex in the graph
    return: path through the graph
    """
    entry = 1
    minheap = [(0, 0, start, 0)] # add first item
    distances = {start: (0, None)} # also the visited set
    end_lat, end_lon = end['lat'], end['lon']

    # need to create node object with parents to find path
    while minheap:
        _, _, curr_node, curr_dist = heapq.heappop(minheap)

        if curr_node == end:
            break

        for out_edge_idx in seattle_graph.incident(curr_node, mode="out"):
            out_edge = seattle_graph.es[out_edge_idx]
            new_dist = curr_dist + out_edge["length"]
            neighbor = seattle_graph.vs.find(id=out_edge["v"])
            dist_to_end = haversine((neighbor['lat'], neighbor['lon']), (end_lat, end_lon)) * 1000 # kilometers to meters

            if neighbor not in distances or new_dist < distances[neighbor][0]:
                distances[neighbor] = (new_dist, curr_node)
            else: # visited
                continue
            
            weight = 2 * (new_dist + dist_to_end)
            heapq.heappush(minheap, (weight, entry, neighbor, new_dist))
            entry += 1


    path = []
    curr = distances[end][1]
    while curr is not None:
        path.append(curr)
        curr = distances[curr][1]

    return path[::-1]


Here is my implementation of the a* search algorithm to find the shortest path between two points in the city of Seattle. Unfortunately my implementation is too slow to be useful in the web application due to the high level nature of python. Because of this I will use an external library (OSMnx) that has a shortest path function.